In [1]:
import anndata as ad
import networkx as nx
import numpy as np
import pandas as pd
import scglue
import seaborn as sns
from IPython import display
from matplotlib import rcParams
from networkx.algorithms.bipartite import biadjacency_matrix
from networkx.drawing.nx_agraph import graphviz_layout

scglue.plot.set_publication_params()
rcParams['figure.figsize'] = (4, 4)


In [2]:
rna = ad.read_h5ad("./data/rna-emb.h5ad")
atac = ad.read_h5ad("./data/atac-emb.h5ad")
guidance_hvf = nx.read_graphml("./data/guidance-hvf.graphml.gz")

In [3]:
rna.var["name"] = rna.var_names
atac.var["name"] = atac.var_names
genes = rna.var.query("highly_variable").index
peaks = atac.var.query("highly_variable").index

In [4]:
features = pd.Index(np.concatenate([rna.var_names, atac.var_names]))
feature_embeddings = np.concatenate([rna.varm["X_glue"], atac.varm["X_glue"]])

skeleton = guidance_hvf.edge_subgraph(
    e for e, attr in dict(guidance_hvf.edges).items()
    if attr["type"] == "fwd"
).copy()

reginf = scglue.genomics.regulatory_inference(
    features, feature_embeddings,
    skeleton=skeleton, random_state=0
)

gene2peak = reginf.edge_subgraph(
    e for e, attr in dict(reginf.edges).items()
    if attr["qval"] < 0.05
)

regulatory_inference:   0%|          | 0/25565 [00:00<?, ?it/s]

In [5]:
scglue.genomics.Bed(atac.var).write_bed("peaks.bed", ncols=3)
scglue.genomics.write_links(
    gene2peak,
    scglue.genomics.Bed(rna.var).strand_specific_start_site(),
    scglue.genomics.Bed(atac.var),
    "gene2peak.links", keep_attrs=["score"]
)
# %conda install -c bioconda pygenometracks

In [6]:
%%writefile tracks.ini

[Score]
file = gene2peak.links
title = Score
height = 2
color = YlGnBu
compact_arcs_level = 2
use_middle = True
file_type = links

[ATAC]
file = peaks.bed
title = ATAC
display = collapsed
border_color = none
labels = False
file_type = bed

[Genes]
file = gencode.vM25.chr_patch_hapl_scaff.annotation.gtf.gz
title = Genes
prefered_name = gene_name
height = 4
merge_transcripts = True
labels = True
max_labels = 100
all_labels_inside = True
style = UCSC
file_type = gtf

[x-axis]
fontsize = 12

Overwriting tracks.ini


In [7]:
loc = rna.var.loc["Figla"]
chrom = loc["chrom"]
chromLen = loc["chromEnd"] - loc["chromStart"]
chromStart = loc["chromStart"] - chromLen
chromEnd = loc["chromEnd"] + chromLen
# !pyGenomeTracks --tracks tracks.ini \
#     --region {chrom}:{chromStart}-{chromEnd} \
#     --outFileName tracks.png 2> /dev/null
!pyGenomeTracks --tracks tracks.ini \
    --region {chrom}:{chromStart}-{chromEnd} \
    --outFileName ./tracks.png 2> ./error
# display.Image("tracks.png")

In [7]:
motif_bed = scglue.genomics.read_bed("../Multiomics-Integration_1/JASPAR2022-mm10.bed.gz")
motif_bed.head()

,chrom,chromStart,chromEnd,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts
0,GL456210.1,159,171,Zbtb6,.,.,.,.,.,.,.,.
1,GL456210.1,242,253,Osr2,.,.,.,.,.,.,.,.
2,GL456210.1,266,278,Pou2f3,.,.,.,.,.,.,.,.
3,GL456210.1,505,517,Eomes,.,.,.,.,.,.,.,.
4,GL456210.1,507,516,Tbr1,.,.,.,.,.,.,.,.


In [8]:
tfs = pd.Index(motif_bed["name"]).intersection(rna.var_names)
tfs.size

532

In [9]:
rna[:, np.union1d(genes, tfs)].write_loom("rna.loom")
np.savetxt("tfs.txt", tfs, fmt="%s")

The loom file will lack these fields:
{'X_glue', 'PCs', 'X_umap', 'X_pca'}
Use write_obsm_varm=True to export multi-dimensional annotations


In [10]:
!pyscenic grn rna.loom tfs.txt \
    -o draft_grn.csv --seed 0 --num_workers 20 \
    --cell_id_attribute cells --gene_attribute genes


2022-10-31 22:47:56,135 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-10-31 22:47:56,401 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork fro

In [11]:
peak_bed = scglue.genomics.Bed(atac.var.loc[peaks])
peak2tf = scglue.genomics.window_graph(peak_bed, motif_bed, 0, right_sorted=True)
peak2tf = peak2tf.edge_subgraph(e for e in peak2tf.edges if e[1] in tfs)

window_graph:   0%|          | 0/25488 [00:00<?, ?it/s]

In [12]:
gene2tf_rank_glue = scglue.genomics.cis_regulatory_ranking(
    gene2peak, peak2tf, genes, peaks, tfs,
    region_lens=atac.var.loc[peaks, "chromEnd"] - atac.var.loc[peaks, "chromStart"],
    random_state=0
)
gene2tf_rank_glue.iloc[:5, :5]

/opt/miniconda3/envs/glue/lib/python3.8/site-packages/scglue/genomics.py:746: FutureWarning: biadjacency_matrix will return a scipy.sparse array instead of a matrix in NetworkX 3.0
  gene2region = biadjacency_matrix(gene2region, genes, regions, dtype=np.int16, weight=None)
/opt/miniconda3/envs/glue/lib/python3.8/site-packages/scglue/genomics.py:747: FutureWarning: biadjacency_matrix will return a scipy.sparse array instead of a matrix in NetworkX 3.0
  region2tf = biadjacency_matrix(region2tf, regions, tfs, dtype=np.int16, weight=None)


cis_reg_ranking.sampling:   0%|          | 0/2000 [00:00<?, ?it/s]

cis_reg_ranking.mapping:   0%|          | 0/1000 [00:00<?, ?it/s]

,Zbtb6,Osr2,Pou2f3,Eomes,Tbr1
genes,,,,,
0610009B22Rik,1193.5,1211.0,1155.5,1115.5,1098.5
1110002J07Rik,1193.5,1211.0,1155.5,1115.5,1098.5
1110006O24Rik,1193.5,1211.0,1155.5,1115.5,1098.5
1110020A21Rik,1193.5,1211.0,1155.5,1115.5,1098.5
1110060G06Rik,1193.5,1211.0,1155.5,1115.5,1098.5


In [13]:
print(type(gene2tf_rank_glue))
gene2tf_rank_glue.to_csv("./gene2tf_rank_glue.csv")

<class 'pandas.core.frame.DataFrame'>


In [14]:
print(type(peak2tf))
nx.write_gml(peak2tf, './peak2tf.gml')

<class 'networkx.classes.multidigraph.MultiDiGraph'>
